In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [2]:
df_jan = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'
df_feb = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet'

In [3]:
# Question 1: Read the data from January. How many columns are there?

df = pd.read_parquet(df_jan)
df.shape

(3066766, 19)

In [4]:
# Question 2: Now let's compute the duration variable. It should contain the duration of a ride in minutes. What's the standard deviation of the trips duration in January?

def read_dataframe(url):

    df = pd.read_parquet(url)
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime']) 
    df['Duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['Duration'] = df.Duration.apply(lambda td: td.total_seconds()/60)
    df['Duration'] = round(df.Duration,2)
    #df = df[(df.Duration >= 1) & (df.Duration <= 60)]
    df.reset_index(drop=True, inplace=True)
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']

    return df

df = read_dataframe(df_jan)
df['Duration'].std()

42.594353179236194

In [5]:
# Question 3: Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).
# What fraction of the records left after you dropped the outliers?

def read_dataframe(url):

    df = pd.read_parquet(url)
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime']) 
    df['Duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['Duration'] = df.Duration.apply(lambda td: td.total_seconds()/60)
    df['Duration'] = round(df.Duration,2)
    df = df[(df.Duration >= 1) & (df.Duration <= 60)]
    df.reset_index(drop=True, inplace=True)
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']

    return df

df = read_dataframe(df_jan)
df.shape

a = 3009173/3066766
a

0.9812202822125979

In [9]:
""" Question 4: Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.
Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)? """

categorical = ['PULocationID','DOLocationID' ]
dv = DictVectorizer()
train_dicts = df[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)
X_train.shape[1]

515

In [10]:
""" Question 5: Now let's use the feature matrix from the previous step to train a model.
Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train? """

target = 'Duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)


7.649261884282595

In [13]:
""" Question 6: Now let's apply this model to the validation dataset (February 2023).
What's the RMSE on validation? """

df_val = read_dataframe(df_feb)
df_val
categorical = ['PULocationID','DOLocationID' ]
target = 'Duration'
dv = DictVectorizer()
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.fit_transform(val_dicts)
y_val = df_val[target].values

lr = LinearRegression()

lr.fit(X_val,y_val)

y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

7.778951220673531